[this doc on github](https://github.com/dotnet/interactive/tree/main/samples/notebooks/csharp/Samples)

# Github repository report

project report for your repository

[Generate a user token](https://help.github.com/en/github/authenticating-to-github/creating-a-personal-access-token-for-the-command-line) to get rid of public [api](https://github.com/octokit/octokit.net/blob/master/docs/getting-started.md) throttling policies for anonymous users 

# The goal is
 * display milestones
 * show milestone burndown
 * what are possible completion dates for milestone
 * milestone work broken down by area


In [1]:
flowchart 
    parameters[Gather org and repo ids] --> creategithubclient[create github client] 
    creategithubclient[create github client]  --> collect[(collect milestones)]
    collect[(collect milestones)] --> collectissues[(collect milestone issues)]
    collectissues[(collect milestone issues)] --> process[process milestone issues]
    process[process milestone issues] --> derive[calculate burndown]
    derive[calculate burndown] --> output[display burndown and milestone work by tag]

In [2]:
#!value --name organization --from-value @input:organization

In [3]:
#!share --from value organization

In [4]:
#!value --name repositoryName --from-value @input:repositoryName

In [5]:
#!share --from value repositoryName 

In [6]:
#!value --name token --from-value @password:github-api-token

In [7]:
#!share --from value token

## Setup
Importing pacakges and setting up connection

In [8]:
#r "nuget: Octokit, 4.0.0"

Installed Packages Octokit, 4.0.0

In [9]:
using Octokit;
using Microsoft.DotNet.Interactive.Formatting;
using Microsoft.DotNet.Interactive.Formatting.TabularData;
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Commands;
using System.Collections.Generic;

In [10]:
plotlyloader = (require.config({
    paths: {
        d3: 'https://cdn.jsdelivr.net/npm/d3@7.4.4/dist/d3.min',
        jquery: 'https://cdn.jsdelivr.net/npm/jquery@3.6.0/dist/jquery.min',
        plotly: 'https://cdn.plot.ly/plotly-2.14.0.min'
    },

    shim: {
        plotly: {
            deps: ['d3', 'jquery'],
            exports: 'plotly'
        }
    }
}) || require);

delay = (delayAmount) => new Promise(resolve => setTimeout(resolve, delayAmount));

In [11]:
var options = new ApiOptions();
var gitHubClient = new GitHubClient(new ProductHeaderValue("notebook"));

if (!string.IsNullOrEmpty(token)) {
    Console.WriteLine("Using github api token");
    var tokenAuth = new Credentials(token);
    gitHubClient.Credentials = tokenAuth;
} else {
    Console.WriteLine("Using anonymous github api");
}

Using github api token


In [12]:
var milestones = (await gitHubClient.Issue.Milestone.GetAllForRepository(organization, repositoryName, options)).Select(m => new{
    Milestone = m,
    Issues = (gitHubClient.Issue.GetAllForRepository(organization, repositoryName, new RepositoryIssueRequest {
        Milestone= m.Number.ToString(),
        State = ItemStateFilter.All
    }, options)).Result.ToArray()
}).ToArray();

In [13]:
var milestoneData = milestones.Select(m =>{
    var lastCountOpen = -1;
    var startDate = m.Milestone.CreatedAt.DateTime;
    var endDate = DateTime.Now.Date;
    var ClosedEveryDay = m.Issues.Where(i => i.ClosedAt.HasValue).GroupBy(i => i.ClosedAt.Value.Date).Select(g => new {Date = g.Key, Count = g.Count()}).OrderBy(e => e.Date).ToArray();
    
    //var OpenedEveryDay = m.Issues.GroupBy(i => i.CreatedAt.Date).Select(g => new {Date = g.Key, Count = g.Count()}).OrderBy(e => e.Date).ToArray();

    //var dates = ClosedEveryDay.Select(e => e.Date).Union(OpenedEveryDay.Select(e => e.Date)).Distinct().OrderBy(d => d).ToArray();
    
    var RollingClosedIssues = ClosedEveryDay.Select(e => new {e.Date, Count = ClosedEveryDay.Where(d => d.Date <= e.Date).Select(d => d.Count).Sum()}).ToArray();
    var RollingOpenIssues = Enumerable.Range(0, (int)(endDate - startDate).TotalDays).Select( i => {
        var date = startDate.AddDays(i);
        var openedCount =  m.Issues.Where(i => (i.ClosedAt.HasValue == false) || (i.ClosedAt > date)).Count();
        return new {Date = date, Count = openedCount};
    }).Where(e => {
        if(e.Count == lastCountOpen){
            return false;
        }else{
            lastCountOpen = e.Count;
            return true;
        }
    }).ToArray();
   
    // OpenedEveryDay.Select(e => new {e.Date, Count = OpenedEveryDay.Where(d => d.Date <= e.Date).Select(d => d.Count).Sum() - ClosedEveryDay.Where(d => d.Date <= e.Date).Select(d => d.Count).Sum()}).ToArray();

    var isClosed = m.Milestone.State.ToString().ToLowerInvariant() == "closed";
    var extrapolations = new List<(DateTime Date, int Count)>();
    var AtRisk = false;
    if(!isClosed){
        var closingIssueSpeed = 0.0;
        var alpha = 0.40;
        for(var i = 0; i < RollingClosedIssues.Length - 1; i++){
            var current = RollingClosedIssues[i];
            var next = RollingClosedIssues[i + 1];
            var days = (next.Date - current.Date).TotalDays;
            if(days > 0){
                var currentSpeed = (double)(next.Count - current.Count) / days;
                closingIssueSpeed = ((1.0-alpha)*currentSpeed) + (alpha*closingIssueSpeed) ;
            }
        }
        closingIssueSpeed = Math.Round(closingIssueSpeed,4, MidpointRounding.AwayFromZero);
        var lastSample = RollingOpenIssues.Last();
        Console.WriteLine($"Milestone {m.Milestone.Title} is {m.Milestone.State}. Closing speed is {closingIssueSpeed} issues per day at {lastSample.Date}.");

        extrapolations = new List<(DateTime Date, int Count)>{
            (lastSample.Date, lastSample.Count)
        };

        // take into account any pause to today

        for(var i = 0; i < (int)((endDate - lastSample.Date).TotalDays); i++){
            var nextCount = lastSample.Count;
            extrapolations.Add((lastSample.Date.AddDays(i), nextCount));
        }

        for(var i = 0; i < extrapolations.Count - 1; i++){
            var current = extrapolations[i];
            var next = extrapolations[i + 1];
            var days = (next.Date - current.Date).TotalDays;
            if(days > 0){
                closingIssueSpeed = alpha*closingIssueSpeed ;
 
            }
        }

        closingIssueSpeed = Math.Round(closingIssueSpeed,2, MidpointRounding.AwayFromZero);
        Console.WriteLine($"Milestone {m.Milestone.Title} is {m.Milestone.State} and has {extrapolations.Count} extrapolated points. Closing speed is {closingIssueSpeed} issues per day.");
         
        var lastExtrapolatedSample = extrapolations.Last();
        var nextSample = lastExtrapolatedSample.Date.AddDays(1);
        var closeDate = lastExtrapolatedSample.Date.AddMonths(1);
        
        if(closingIssueSpeed > 0){
            var daysToClose = lastExtrapolatedSample.Count / closingIssueSpeed;
            closeDate = lastExtrapolatedSample.Date.AddDays(daysToClose);
            Console.WriteLine($"Milestone {m.Milestone.Title} is {m.Milestone.State} will be closed by {closeDate}.");
        }else{
            AtRisk = true;
            Console.WriteLine($"Milestone {m.Milestone.Title} will not be closed anytime soon.");
        }

        var lastCount = lastExtrapolatedSample.Count;
        while(nextSample < closeDate){
            lastCount -= (int)(closingIssueSpeed);
            extrapolations.Add((nextSample, lastCount));
            nextSample = nextSample.AddDays(1);            
        }
    }

    return new {
        m.Milestone,
        m.Issues,
        ClosedEveryDay,
       // OpenedEveryDay,
        RollingClosedIssues,
        RollingOpenIssues,
        AtRisk,
        ToComplete = extrapolations.Select(e => new {e.Date, e.Count}).ToArray()
    };
    }).ToArray();

Milestone VS Code Extension GA is open. Closing speed is 0.8558 issues per day at 11/10/2022 21:40:38.
Milestone VS Code Extension GA is open and has 17 extrapolated points. Closing speed is 0 issues per day.
Milestone VS Code Extension GA will not be closed anytime soon.
Milestone API Stabilization is open. Closing speed is 0 issues per day at 03/08/2022 15:55:09.
Milestone API Stabilization is open and has 86 extrapolated points. Closing speed is 0 issues per day.
Milestone API Stabilization will not be closed anytime soon.


In [ ]:
milestoneData.Select(m => new {m.Milestone.Title, m.Milestone.Description, m.Milestone.DueOn, m.Milestone.ClosedAt, m.Milestone.State,m.Milestone.OpenIssues, m.Milestone.ClosedIssues, m.Milestone.CreatedAt, m.AtRisk}).ToTabularDataResource().Display();

In [14]:
var milestoneBurndown =  milestoneData.Where(m => m.Milestone.State == "Open")
.OrderByDescending(m => m.Milestone.CreatedAt)
.Select(m => new { Title = m.Milestone.Title, OpenIssues = m.RollingOpenIssues.ToArray(), ToComplete = m.ToComplete.ToArray(), m.AtRisk}).ToArray();

In [15]:
#!share --from csharp milestoneBurndown

In [16]:
<div id="target"></div>

In [17]:
  const traces = [];

  const layout = {
      title: 'Milestone Burndown',
      grid: { rows: milestoneBurndown.length, columns: 1, pattern: 'independent' },
      annotations: []
    };

    for(let i = 0; i < milestoneBurndown.length; i++) {       
        layout[`xaxis${i+1}}`] = {};
        layout[`yaxis${i+1}`] = { title: "Open items" };
        const milestone = milestoneBurndown[i];        
        const done = {
            y: Array.from(milestone.OpenIssues.map(x => x.Count)),
            x: Array.from(milestone.OpenIssues.map(x => x.Date)),
            mode: 'lines',
            //name: `Done [${milestone.Title}]`,
            line: {
              dash: 'solid',
              width: 4
            },
            xaxis: `x${i+1}`,
            yaxis: `y${i+1}`,
            //hovertemplate: `<b>${milestone.Title}</b><br><i>Issue count</i>: %{y}<br><b>Date</b>: %{x}<extra></extra>`,
            type: 'scattergl'
        };

        const toDo = {
            y: Array.from(milestone.ToComplete.map(x => x.Count)),
            x: Array.from(milestone.ToComplete.map(x => x.Date)),
            mode: 'lines',
            //name: `To Do [${milestone.Title}]`,
            line: {
              dash: 'dashdot',
              width: 4
            },
            xaxis: `x${i+1}`,
            yaxis: `y${i+1}`,
            //hovertemplate: `<b>${milestone.Title} Projection</b>}<br><i>Issue count</i>: %{y}<br><b>Date</b>: %{x}<extra></extra>`,
            type: 'scattergl'
        };

        layout.annotations.push({
          x: done.x[0],
          y: done.y.reduce((max, value) => {return Math.max(max, value)}),
          yshift: 10 + done.line.width,
          xanchor: 'left',
          xref: `x${i+1}`,
          yref: `y${i+1}`,
          text: milestone.Title,
          showarrow: false
        });
        
        if(milestone.AtRisk) {
          layout.annotations.push({
            x: done.x[done.x.length - 1],
            y: done.y[done.y.length - 1],
            xanchor: 'center',
            yanchor: 'bottom',
            align: 'center',
            xref: `x${i+1}`,
            yref: `y${i+1}`,
            text: "\u26A0",
            showarrow: true,
            ax: 0,
            ay: -(20 + done.line.width),
            font: {
              color: "red",
              size: 30
            }
          });
        }

        traces.push(done);
        traces.push(toDo);
    }
    
exportData = { traces, layout };

plotlyloader(['d3', 'plotly'], function (d3, plotly) {
  console.log("Plotly loaded"); 
  plotly.newPlot('target', exportData.traces, exportData.layout, {responsive: true});
});


In [ ]:
using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.Commands;

void PieWithMermaid(IEnumerable<(string area,int doneCount)> data, string label){
    double total = data.Select(d => d.doneCount).Sum();
    var slices = data.Select(d => $"    \"{d.area}\" : {Math.Round((d.doneCount/total)*100.0, 2)}").ToArray();

    var mermaidPieMarkdown = new StringBuilder();
    mermaidPieMarkdown.AppendLine("pie");
    mermaidPieMarkdown.AppendLine($@"    title {label}");

    foreach(var slice in slices){
        mermaidPieMarkdown.AppendLine(slice);
    }
    Kernel.Root.SendAsync(new SendEditableCode("mermaid",mermaidPieMarkdown.ToString()));
}

In [ ]:

foreach(var milestone in milestoneData.OrderBy(m => m.Milestone.CreatedAt)){
    var doneIssues = milestone.Issues.Where(i => i.ClosedAt.HasValue).ToArray();
    if(doneIssues.Length > 0) {
        var doneData = doneIssues.SelectMany( i => i.Labels.Select(l => l.Name)).Where(l => l.StartsWith("Area-")).GroupBy(l => l).Select(l => (l.Key,l.Count()));
        PieWithMermaid(doneData, $"Milestone: {milestone.Milestone.Title} work done by tag ({doneIssues.Length} of { milestone.Issues.Count()} items)");
    }
   
    var toDoIssues = milestone.Issues.Where(i => i.ClosedAt.HasValue == false).ToArray();
    if(toDoIssues.Length > 0) {
        var toDoData = toDoIssues.SelectMany( i => i.Labels.Select(l => l.Name)).Where(l => l.StartsWith("Area-")).GroupBy(l => l).Select(l => (l.Key,l.Count()));
        PieWithMermaid(toDoData, $"Milestone: {milestone.Milestone.Title} work to do  by tag({toDoIssues.Length} of { milestone.Issues.Count()} items)");
    }
}

In [ ]:
pie
    title Milestone: VS Code Extension GA work to do  by tag(15 of 38 items)
    "Area-VS Code Extension" : 66.67
    "Area-VS Code Jupyter Extension Interop" : 5.56
    "Area-Documentation" : 5.56
    "Area-Getting Started" : 16.67
    "Area-Formatting" : 5.56


In [ ]:
pie
    title Milestone: API Stabilization work to do  by tag(8 of 8 items)
    "Area-Documentation" : 10
    "Area-Getting Started" : 10
    "Area-VS Code Extension" : 10
    "Area-Messaging and Comms" : 10
    "Area-Packages and Extensions" : 10
    "Area-SQL / data querying" : 20
    "Area-F#" : 10
    "Area-Language Services" : 20


In [ ]:
pie
    title Milestone: VS Code Extension GA work done by tag (23 of 38 items)
    "Area-Formatting" : 4.76
    "Area-Language Services" : 4.76
    "Area-Packages and Extensions" : 4.76
    "Area-Telemetry" : 9.52
    "Area-Messaging and Comms" : 9.52
    "Area-VS Code Extension" : 42.86
    "Area-Variable sharing" : 14.29
    "Area-Performance" : 4.76
    "Area-JavaScript HTML CSS" : 4.76


In [ ]:
#!share --from javascript exportData
System.IO.File.WriteAllText("c:\\temp\\traces.json", exportData.RootElement.ToDisplayString());

In [103]:
const d = {
    "traces": [
        {
            "y": [
                8
            ],
            "x": [
                "2022-08-03T15:55:09"
            ],
            "mode": "lines",
            "name": "Done [API Stabilization]",
            "line": {
                "dash": "solid",
                "width": 4
            },
            "xaxis": "x1",
            "yaxis": "y1",
            "hovertemplate": "<b>API Stabilization</b><br><i>Issue count</i>: %{y}<br><b>Date</b>: %{x}<extra></extra>",
            "type": "scattergl"
        },
        {
            "y": [
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8,
                8
            ],
            "x": [
                "2022-08-03T15:55:09",
                "2022-08-03T15:55:09",
                "2022-08-04T15:55:09",
                "2022-08-05T15:55:09",
                "2022-08-06T15:55:09",
                "2022-08-07T15:55:09",
                "2022-08-08T15:55:09",
                "2022-08-09T15:55:09",
                "2022-08-10T15:55:09",
                "2022-08-11T15:55:09",
                "2022-08-12T15:55:09",
                "2022-08-13T15:55:09",
                "2022-08-14T15:55:09",
                "2022-08-15T15:55:09",
                "2022-08-16T15:55:09",
                "2022-08-17T15:55:09",
                "2022-08-18T15:55:09",
                "2022-08-19T15:55:09",
                "2022-08-20T15:55:09",
                "2022-08-21T15:55:09",
                "2022-08-22T15:55:09",
                "2022-08-23T15:55:09",
                "2022-08-24T15:55:09",
                "2022-08-25T15:55:09",
                "2022-08-26T15:55:09",
                "2022-08-27T15:55:09",
                "2022-08-28T15:55:09",
                "2022-08-29T15:55:09",
                "2022-08-30T15:55:09",
                "2022-08-31T15:55:09",
                "2022-09-01T15:55:09",
                "2022-09-02T15:55:09",
                "2022-09-03T15:55:09",
                "2022-09-04T15:55:09",
                "2022-09-05T15:55:09",
                "2022-09-06T15:55:09",
                "2022-09-07T15:55:09",
                "2022-09-08T15:55:09",
                "2022-09-09T15:55:09",
                "2022-09-10T15:55:09",
                "2022-09-11T15:55:09",
                "2022-09-12T15:55:09",
                "2022-09-13T15:55:09",
                "2022-09-14T15:55:09",
                "2022-09-15T15:55:09",
                "2022-09-16T15:55:09",
                "2022-09-17T15:55:09",
                "2022-09-18T15:55:09",
                "2022-09-19T15:55:09",
                "2022-09-20T15:55:09",
                "2022-09-21T15:55:09",
                "2022-09-22T15:55:09",
                "2022-09-23T15:55:09",
                "2022-09-24T15:55:09",
                "2022-09-25T15:55:09",
                "2022-09-26T15:55:09",
                "2022-09-27T15:55:09",
                "2022-09-28T15:55:09",
                "2022-09-29T15:55:09",
                "2022-09-30T15:55:09",
                "2022-10-01T15:55:09",
                "2022-10-02T15:55:09",
                "2022-10-03T15:55:09",
                "2022-10-04T15:55:09",
                "2022-10-05T15:55:09",
                "2022-10-06T15:55:09",
                "2022-10-07T15:55:09",
                "2022-10-08T15:55:09",
                "2022-10-09T15:55:09",
                "2022-10-10T15:55:09",
                "2022-10-11T15:55:09",
                "2022-10-12T15:55:09",
                "2022-10-13T15:55:09",
                "2022-10-14T15:55:09",
                "2022-10-15T15:55:09",
                "2022-10-16T15:55:09",
                "2022-10-17T15:55:09",
                "2022-10-18T15:55:09",
                "2022-10-19T15:55:09",
                "2022-10-20T15:55:09",
                "2022-10-21T15:55:09",
                "2022-10-22T15:55:09",
                "2022-10-23T15:55:09",
                "2022-10-24T15:55:09",
                "2022-10-25T15:55:09",
                "2022-10-26T15:55:09",
                "2022-10-27T15:55:09",
                "2022-10-28T15:55:09",
                "2022-10-29T15:55:09",
                "2022-10-30T15:55:09",
                "2022-10-31T15:55:09",
                "2022-11-01T15:55:09",
                "2022-11-02T15:55:09",
                "2022-11-03T15:55:09",
                "2022-11-04T15:55:09",
                "2022-11-05T15:55:09",
                "2022-11-06T15:55:09",
                "2022-11-07T15:55:09",
                "2022-11-08T15:55:09",
                "2022-11-09T15:55:09",
                "2022-11-10T15:55:09",
                "2022-11-11T15:55:09",
                "2022-11-12T15:55:09",
                "2022-11-13T15:55:09",
                "2022-11-14T15:55:09",
                "2022-11-15T15:55:09",
                "2022-11-16T15:55:09",
                "2022-11-17T15:55:09",
                "2022-11-18T15:55:09",
                "2022-11-19T15:55:09",
                "2022-11-20T15:55:09",
                "2022-11-21T15:55:09",
                "2022-11-22T15:55:09",
                "2022-11-23T15:55:09",
                "2022-11-24T15:55:09",
                "2022-11-25T15:55:09"
            ],
            "mode": "lines",
            "name": "To Do [API Stabilization]",
            "line": {
                "dash": "dashdot",
                "width": 4
            },
            "xaxis": "x1",
            "yaxis": "y1",
            "hovertemplate": "<b>API Stabilization Projection</b>}<br><i>Issue count</i>: %{y}<br><b>Date</b>: %{x}<extra></extra>",
            "type": "scattergl"
        },
        {
            "y": [
                38,
                36,
                35,
                34,
                33,
                32,
                31,
                30,
                29,
                28,
                27,
                26,
                25,
                24,
                23,
                21,
                20,
                18,
                17,
                16,
                15
            ],
            "x": [
                "2021-10-05T21:40:38",
                "2021-11-15T21:40:38",
                "2022-03-15T21:40:38",
                "2022-05-18T21:40:38",
                "2022-06-07T21:40:38",
                "2022-06-09T21:40:38",
                "2022-06-14T21:40:38",
                "2022-06-27T21:40:38",
                "2022-07-08T21:40:38",
                "2022-07-15T21:40:38",
                "2022-07-26T21:40:38",
                "2022-07-27T21:40:38",
                "2022-09-06T21:40:38",
                "2022-09-15T21:40:38",
                "2022-09-21T21:40:38",
                "2022-10-04T21:40:38",
                "2022-10-05T21:40:38",
                "2022-10-06T21:40:38",
                "2022-10-07T21:40:38",
                "2022-10-08T21:40:38",
                "2022-10-11T21:40:38"
            ],
            "mode": "lines",
            "name": "Done [VS Code Extension GA]",
            "line": {
                "dash": "solid",
                "width": 4
            },
            "xaxis": "x2",
            "yaxis": "y2",
            "hovertemplate": "<b>VS Code Extension GA</b><br><i>Issue count</i>: %{y}<br><b>Date</b>: %{x}<extra></extra>",
            "type": "scattergl"
        },
        {
            "y": [
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15,
                15
            ],
            "x": [
                "2022-10-11T21:40:38",
                "2022-10-11T21:40:38",
                "2022-10-12T21:40:38",
                "2022-10-13T21:40:38",
                "2022-10-14T21:40:38",
                "2022-10-15T21:40:38",
                "2022-10-16T21:40:38",
                "2022-10-17T21:40:38",
                "2022-10-18T21:40:38",
                "2022-10-19T21:40:38",
                "2022-10-20T21:40:38",
                "2022-10-21T21:40:38",
                "2022-10-22T21:40:38",
                "2022-10-23T21:40:38",
                "2022-10-24T21:40:38",
                "2022-10-25T21:40:38",
                "2022-10-26T21:40:38",
                "2022-10-27T21:40:38",
                "2022-10-28T21:40:38",
                "2022-10-29T21:40:38",
                "2022-10-30T21:40:38",
                "2022-10-31T21:40:38",
                "2022-11-01T21:40:38",
                "2022-11-02T21:40:38",
                "2022-11-03T21:40:38",
                "2022-11-04T21:40:38",
                "2022-11-05T21:40:38",
                "2022-11-06T21:40:38",
                "2022-11-07T21:40:38",
                "2022-11-08T21:40:38",
                "2022-11-09T21:40:38",
                "2022-11-10T21:40:38",
                "2022-11-11T21:40:38",
                "2022-11-12T21:40:38",
                "2022-11-13T21:40:38",
                "2022-11-14T21:40:38",
                "2022-11-15T21:40:38",
                "2022-11-16T21:40:38",
                "2022-11-17T21:40:38",
                "2022-11-18T21:40:38",
                "2022-11-19T21:40:38",
                "2022-11-20T21:40:38",
                "2022-11-21T21:40:38",
                "2022-11-22T21:40:38",
                "2022-11-23T21:40:38",
                "2022-11-24T21:40:38",
                "2022-11-25T21:40:38"
            ],
            "mode": "lines",
            "name": "To Do [VS Code Extension GA]",
            "line": {
                "dash": "dashdot",
                "width": 4
            },
            "xaxis": "x2",
            "yaxis": "y2",
            "hovertemplate": "<b>VS Code Extension GA Projection</b>}<br><i>Issue count</i>: %{y}<br><b>Date</b>: %{x}<extra></extra>",
            "type": "scattergl"
        }
    ],
    "layout": {
        "title": "Milestone Burndown",
        "grid": {
            "rows": 2,
            "columns": 1,
            "pattern": "independent"
        },
        "annotations": [
            {
                "x": "2022-08-03T15:55:09",
                "y": 8,
                "yshift": 14,
                "xanchor": "left",
                "xref": "x1",
                "yref": "y1",
                "text": "API Stabilization",
                "showarrow": false
            },
            {
                "x": "2022-08-03T15:55:09",
                "y": 8,
                "xanchor": "center",
                "yanchor": "bottom",
                "align": "center",
                "xref": "x1",
                "yref": "y1",
                "text": "⚠",
                "showarrow": true,
                "ax": 0,
                "ay": -24,
                "font": {
                    "color": "red",
                    "size": 30
                }
            },
            {
                "x": "2021-10-05T21:40:38",
                "y": 38,
                "yshift": 14,
                "xanchor": "left",
                "xref": "x2",
                "yref": "y2",
                "text": "VS Code Extension GA",
                "showarrow": false
            },
            {
                "x": "2022-10-11T21:40:38",
                "y": 15,
                "xanchor": "center",
                "yanchor": "bottom",
                "align": "center",
                "xref": "x2",
                "yref": "y2",
                "text": "⚠",
                "showarrow": true,
                "ax": 0,
                "ay": -24,
                "font": {
                    "color": "red",
                    "size": 30
                }
            }
        ],
        "xaxis1}": {},
        "yaxis1": {
            "title": "Open items"
        },
        "xaxis2}": {},
        "yaxis2": {
            "title": "Open items"
        }
    }
};

plotlyloader(['d3', 'plotly'], function (d3, plotly) {
    console.log("Plotly loaded"); 
    plotly.newPlot('target', d.traces, d.layout, {responsive: true});
});